In [22]:
from neuralforecast.utils import AirPassengersDF
import pandas as pd
from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS
import matplotlib.pyplot as plt
from neuralforecast.losses.pytorch import MQLoss
from utilsforecast.plotting import plot_series
import fn
from toolz.functoolz import pipe
from matplotlib import cm
import matplotlib
import plotly.graph_objects as go

In [40]:
# Y_df = AirPassengersDF # Defined in neuralforecast.utils
# Y_df.head()

Y_df = pd.read_csv("./testData/test.csv",dtype={'ds': 'U'},)
Y_df['ds'] = pd.to_datetime(Y_df['ds'], format='%Y-%m-%dT%H:%M:%SZ')
Y_df.head()

Y_df.dtypes

unique_id            object
ds           datetime64[ns]
y                   float64
dtype: object

In [43]:
horizon = 2
levels = [80,90,99]
# Try different hyperparmeters to improve accuracy.
models = [NHITS(h=horizon,                      # Forecast horizon
                input_size=2 * horizon,         # Length of input sequence
                max_steps=10,                 # Number of steps to train
                n_freq_downsample=[2, 1, 1],    # Downsampling factors for each stack output
                mlp_units = 3 * [[1024, 1024]],
                  loss=MQLoss(level=levels),
                  
                ) # Number of units in each block.
          ]
nf = NeuralForecast(models=models, freq='M')


Seed set to 1


In [44]:
nf.fit(df=Y_df, val_size=horizon)

/home/philip/miniconda3/envs/prophetTest/lib/python3.11/site-packages/neuralforecast/common/_base_model.py:208: UserWarning: val_check_steps is greater than max_steps, setting val_check_steps to max_steps.
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name         | Type          | Params
-----------------------------------------------
0 | loss         | MQLoss        | 7     
1 | padder_train | ConstantPad1d | 0     
2 | scaler       | TemporalNorm  | 0     
3 | blocks       | ModuleList    | 9.5 M 
-----------------------------------------------
9.5 M     Trainable params
7         Non-trainable params
9.5 M     Total params
38.023    Total estimated model params size (MB)


Epoch 9: 100%|██████████| 1/1 [00:00<00:00,  8.93it/s, v_num=110, train_loss_step=0.0413, train_loss_epoch=0.0413, valid_loss=0.112]

`Trainer.fit` stopped: `max_steps=10` reached.


Epoch 9: 100%|██████████| 1/1 [00:00<00:00,  8.76it/s, v_num=110, train_loss_step=0.0413, train_loss_epoch=0.0413, valid_loss=0.112]


In [47]:
Y_hat_insample = nf.predict_insample(step_size=horizon)
Y_hat_insample.head()

/home/philip/miniconda3/envs/prophetTest/lib/python3.11/site-packages/utilsforecast/processing.py:374: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 98.10it/s] 


/home/philip/miniconda3/envs/prophetTest/lib/python3.11/site-packages/neuralforecast/core.py:196: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


,ds,cutoff,NHITS-median,NHITS-lo-99,NHITS-lo-90,NHITS-lo-80,NHITS-hi-80,NHITS-hi-90,NHITS-hi-99,y
unique_id,,,,,,,,,,
a,2024-01-01 12:00:00,2023-12-31 12:00:00,-0.108353,-0.407957,-0.338368,-0.150701,0.064993,0.007472,0.273509,1.2
a,2024-01-01 13:00:00,2023-12-31 12:00:00,-0.185011,-0.476949,-0.243333,-0.120072,0.037656,0.044760,-0.063932,1.3
a,2024-01-01 14:00:00,2023-12-31 14:00:00,0.984654,0.255963,0.574604,0.940408,1.537441,1.361575,1.836984,1.7
a,2024-01-01 15:00:00,2023-12-31 14:00:00,0.923275,0.192768,0.694973,0.953472,1.496655,1.410971,1.442286,1.6
a,2024-01-01 16:00:00,2023-12-31 16:00:00,1.289634,0.059408,0.607347,0.950567,1.919309,1.771615,2.333704,1.2


In [48]:
df = Y_hat_insample.reset_index()
df = df.drop(['cutoff','y'],axis=1)
df = df.rename(columns={"NHITS-median": "NHITS"})
df


,unique_id,ds,NHITS,NHITS-lo-99,NHITS-lo-90,NHITS-lo-80,NHITS-hi-80,NHITS-hi-90,NHITS-hi-99
0,a,2024-01-01 12:00:00,-0.108353,-0.407957,-0.338368,-0.150701,0.064993,0.007472,0.273509
1,a,2024-01-01 13:00:00,-0.185011,-0.476949,-0.243333,-0.120072,0.037656,0.044760,-0.063932
2,a,2024-01-01 14:00:00,0.984654,0.255963,0.574604,0.940408,1.537441,1.361575,1.836984
3,a,2024-01-01 15:00:00,0.923275,0.192768,0.694973,0.953472,1.496655,1.410971,1.442286
4,a,2024-01-01 16:00:00,1.289634,0.059408,0.607347,0.950567,1.919309,1.771615,2.333704
5,a,2024-01-01 17:00:00,1.137796,-0.009922,0.658496,1.012793,1.882055,1.745244,2.052947
6,a,2024-01-01 18:00:00,1.132224,-0.196516,0.319200,0.660871,1.584400,1.482992,2.039933
7,a,2024-01-01 19:00:00,0.853178,-0.250263,0.372104,0.713475,1.545412,1.433161,1.727971
8,b,2024-01-01 12:00:00,-0.108353,-0.407957,-0.338368,-0.150701,0.064993,0.007472,0.273509
9,b,2024-01-01 13:00:00,-0.185011,-0.476949,-0.243333,-0.120072,0.037656,0.044760,-0.063932


In [50]:
#p=matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","violet","blue"])
p = list(["red","blue"])
fig = plot_series(Y_df, df, level=[90], engine="plotly")
# fill='tonexty',  fill='tonexty',
fig.get_subplot
# row=1, col=1
for r in [ 1,2]: 
  fig.add_trace(go.Scatter(x=df['ds'], y=df['NHITS-lo-99'],row=1, col=r,  mode='lines', line=dict(color='lightgrey'), name='99% Prediction Interval'))
  fig.add_trace(go.Scatter(x=df['ds'], y=df['NHITS-hi-99'],row=1, col=r,  mode='lines', line=dict(color='lightgrey'), name='99% Prediction Interval'))



fig.show()

In [15]:
dir(cm)

['Accent',
 'Accent_r',
 'Blues',
 'Blues_r',
 'BrBG',
 'BrBG_r',
 'BuGn',
 'BuGn_r',
 'BuPu',
 'BuPu_r',
 'CMRmap',
 'CMRmap_r',
 'ColormapRegistry',
 'Dark2',
 'Dark2_r',
 'GnBu',
 'GnBu_r',
 'Grays',
 'Greens',
 'Greens_r',
 'Greys',
 'Greys_r',
 'Mapping',
 'OrRd',
 'OrRd_r',
 'Oranges',
 'Oranges_r',
 'PRGn',
 'PRGn_r',
 'Paired',
 'Paired_r',
 'Pastel1',
 'Pastel1_r',
 'Pastel2',
 'Pastel2_r',
 'PiYG',
 'PiYG_r',
 'PuBu',
 'PuBuGn',
 'PuBuGn_r',
 'PuBu_r',
 'PuOr',
 'PuOr_r',
 'PuRd',
 'PuRd_r',
 'Purples',
 'Purples_r',
 'RdBu',
 'RdBu_r',
 'RdGy',
 'RdGy_r',
 'RdPu',
 'RdPu_r',
 'RdYlBu',
 'RdYlBu_r',
 'RdYlGn',
 'RdYlGn_r',
 'Reds',
 'Reds_r',
 'ScalarMappable',
 'Set1',
 'Set1_r',
 'Set2',
 'Set2_r',
 'Set3',
 'Set3_r',
 'Spectral',
 'Spectral_r',
 'Wistia',
 'Wistia_r',
 'YlGn',
 'YlGnBu',
 'YlGnBu_r',
 'YlGn_r',
 'YlOrBr',
 'YlOrBr_r',
 'YlOrRd',
 'YlOrRd_r',
 '_LUTSIZE',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '